In [1]:
import imageio 
import torch
from PIL import Image
from torchvision import transforms
import numpy as np 
import matplotlib.pyplot as plt 
from ptflops import get_model_complexity_info


C:\Users\tycha\anaconda3\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\tycha\anaconda3\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
import pandas as pd
housing = pd.DataFrame(pd.read_csv("Housing.csv"))
housing.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [3]:
num_vars = ['area', 'bedrooms', 'bathrooms', 'stories', 'parking', 'price'] 
nNetwork = housing[num_vars] 
nNetwork.head() 


,area,bedrooms,bathrooms,stories,parking,price
0,7420,4,2,3,2,13300000
1,8960,4,4,4,3,12250000
2,9960,3,2,2,2,12250000
3,7500,4,2,2,3,12215000
4,7420,4,1,2,2,11410000


In [4]:
#Price Predicition
area = torch.tensor(nNetwork['area']) 
bedrooms = torch.tensor(nNetwork['bedrooms'])
bathrooms = torch.tensor(nNetwork['bathrooms'])
stories = torch.tensor(nNetwork['stories'])
parking = torch.tensor(nNetwork['parking'])
price = torch.tensor(nNetwork['price'])

#Normalized #Split Post Normalized Values
normArea = area / max(nNetwork['area'])
normBedrooms = bedrooms / max(nNetwork['bedrooms'])
normBathrooms = bathrooms / max(nNetwork['bathrooms'])
normStories = stories / max(nNetwork['stories'])
normParking = parking / max(nNetwork['parking'])
normPrice = price / max(nNetwork['price'])

#Split ^
#Mask Creation 80/20 Split
mask = np.random.rand(len(normArea)) <= 0.8

#AreaSets
tArea = normArea[mask]
vArea = normArea[~mask]

#BedroomSets
tBedrooms = normBedrooms[mask]
vBedrooms = normBedrooms[~mask]

#BathroomSets
tBathrooms = normBathrooms[mask]
vBathrooms = normBathrooms[~mask]

#StoriesSets
tStories = normStories[mask]
vStories = normStories[~mask]

#ParkingSets
tParking = normParking[mask]
vParking = normParking[~mask]

#PriceSets
tPrice = normPrice[mask]
vPrice = normPrice[~mask]

In [5]:
#Bringing the normalized features back together and making it a tensor.
tCaterogies = [tArea, tBedrooms, tBathrooms, tStories, tParking]
tCaterogies = torch.stack(tCaterogies)
tCaterogies = np.transpose(tCaterogies)

vCaterogies = [vArea, vBedrooms, vBathrooms, vStories, vParking]
vCaterogies = torch.stack(vCaterogies)
vCaterogies = np.transpose(vCaterogies)

In [6]:
tCaterogies

tensor([[0.4580, 0.6667, 0.5000, 0.7500, 0.6667],
        [0.5531, 0.6667, 1.0000, 1.0000, 1.0000],
        [0.4630, 0.6667, 0.5000, 0.5000, 1.0000],
        ...,
        [0.2235, 0.3333, 0.2500, 0.2500, 0.0000],
        [0.1796, 0.5000, 0.2500, 0.2500, 0.0000],
        [0.2377, 0.5000, 0.2500, 0.5000, 0.0000]])

In [7]:
vCaterogies

tensor([[0.6148, 0.5000, 0.5000, 0.5000, 0.6667],
        [0.3704, 0.6667, 0.7500, 0.5000, 0.6667],
        [0.4037, 0.6667, 0.5000, 0.5000, 0.6667],
        [0.3679, 0.5000, 0.7500, 0.5000, 0.3333],
        [0.3704, 0.5000, 0.5000, 0.7500, 0.0000],
        [0.3704, 0.6667, 0.5000, 1.0000, 0.3333],
        [0.7062, 0.6667, 0.2500, 0.5000, 0.3333],
        [0.4741, 0.6667, 0.5000, 1.0000, 0.3333],
        [0.6898, 0.5000, 0.2500, 0.2500, 0.3333],
        [0.3099, 0.5000, 0.2500, 1.0000, 0.0000],
        [0.3704, 0.5000, 0.5000, 1.0000, 0.0000],
        [0.4117, 0.5000, 0.2500, 0.7500, 0.0000],
        [0.3704, 0.6667, 0.2500, 0.7500, 0.0000],
        [0.2778, 0.5000, 0.2500, 1.0000, 0.0000],
        [0.4083, 0.6667, 0.5000, 0.5000, 0.3333],
        [0.4074, 0.5000, 0.2500, 0.2500, 0.6667],
        [0.4037, 0.5000, 0.2500, 0.2500, 0.6667],
        [0.3860, 0.6667, 0.5000, 0.2500, 0.3333],
        [0.4012, 0.5000, 0.5000, 0.7500, 0.0000],
        [0.3395, 0.5000, 0.2500, 0.7500, 0.3333],


In [8]:
# Sequential Model Setup 
import torch.nn as nn
import torch.optim as optim
model = nn.Sequential(
                nn.Linear(5,8), # (X,Y) X = Input Features Y = Output Features
                nn.Tanh(),  # Neuron Activiation Function
                nn.Linear(8,16), # Hidden Layer 
                nn.Tanh(),
                nn.Linear(16,1)) # End Layer (Y,N) # Output Features as Input, N - Price Output
model


Sequential(
  (0): Linear(in_features=5, out_features=8, bias=True)
  (1): Tanh()
  (2): Linear(in_features=8, out_features=16, bias=True)
  (3): Tanh()
  (4): Linear(in_features=16, out_features=1, bias=True)
)

In [9]:
#Optimizer Initialization
OPTIM = optim.SGD(model.parameters(), lr = 1e-3)

In [10]:
#Training Loop Function Declaration
def training_loop(numOfEpochs, optimizer, model, lossFunction, inputTrainCater, inputValidCater, outputTrainPrice, outputValidPrice):
    for epoch in range(1, numOfEpochs + 1):
        tTrain = model(inputTrainCater)
        trainLoss = lossFunction(tTrain, outputTrainPrice)
        
        vTrain = model(inputValidCater)
        valLoss = lossFunction(vTrain, outputValidPrice)

        optimizer.zero_grad()
        trainLoss.backward()
        optimizer.step()

        if epoch == 200:
            print(f"Epoch {epoch}, Training loss {trainLoss.item():0.4f},"f" Validation loss {valLoss.item():.4f}")

In [11]:
#Training Loop Intialization
training_loop(
    numOfEpochs = 200,
    optimizer = OPTIM,
    model = model,
    lossFunction = nn.MSELoss(),
    inputTrainCater = tCaterogies,
    inputValidCater = vCaterogies,
    outputTrainPrice = tPrice,
    outputValidPrice = vPrice)

C:\Users\tycha\anaconda3\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([445])) that is different to the input size (torch.Size([445, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\tycha\anaconda3\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 200, Training loss 0.0485, Validation loss 0.0407


In [12]:
with torch.cuda.device(0):
  net = model
  macs, paramaters = get_model_complexity_info(net, (1, 5), as_strings=True, print_per_layer_stat=True, verbose=True)
  print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
  print('{:<30}  {:<8}'.format('Number of parameters: ', paramaters))

Sequential(
  0.0 M, 100.000% Params, 0.0 GMac, 100.000% MACs, 
  (0): Linear(0.0 M, 22.967% Params, 0.0 GMac, 22.967% MACs, in_features=5, out_features=8, bias=True)
  (1): Tanh(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, )
  (2): Linear(0.0 M, 68.900% Params, 0.0 GMac, 68.900% MACs, in_features=8, out_features=16, bias=True)
  (3): Tanh(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, )
  (4): Linear(0.0 M, 8.134% Params, 0.0 GMac, 8.134% MACs, in_features=16, out_features=1, bias=True)
)
Computational complexity:       0.0 GMac
Number of parameters:           209     


In [13]:
############ Part 2 ################

In [14]:
# Sequential Model Setup 
modelP2 = nn.Sequential(
                nn.Linear(5,8), # (X,Y) X = Input Features Y = Output Features
                nn.Tanh(),  # Neuron Activiation Function
                nn.Linear(8,16), # Hidden Layer 
                nn.Tanh(),
                nn.Linear(16, 32),
                nn.Tanh(),
                nn.Linear(32,64),
                nn.Tanh(),
                nn.Linear(64,1)) # End Layer (Y,N) # Output Features as Input, N - Price Output
modelP2

Sequential(
  (0): Linear(in_features=5, out_features=8, bias=True)
  (1): Tanh()
  (2): Linear(in_features=8, out_features=16, bias=True)
  (3): Tanh()
  (4): Linear(in_features=16, out_features=32, bias=True)
  (5): Tanh()
  (6): Linear(in_features=32, out_features=64, bias=True)
  (7): Tanh()
  (8): Linear(in_features=64, out_features=1, bias=True)
)

In [15]:
#Optimizer Initialization
OPTIM2 = optim.SGD(modelP2.parameters(), lr = 1e-3)

In [16]:
#Training Loop Function Declaration
def training_loop(numOfEpochs, optimizer, model, lossFunction, inputTrainCater, inputValidCater, outputTrainPrice, outputValidPrice):
    for epoch in range(1, numOfEpochs + 1):
        tTrain = model(inputTrainCater)
        trainLoss = lossFunction(tTrain, outputTrainPrice)
        
        vTrain = model(inputValidCater)
        valLoss = lossFunction(vTrain, outputValidPrice)

        optimizer.zero_grad()
        trainLoss.backward()
        optimizer.step()

        if epoch == 200:
            print(f"Epoch {epoch}, Training loss {trainLoss.item():0.4f},"f" Validation loss {valLoss.item():.4f}")

In [17]:
#Training Loop Intialization
training_loop(
    numOfEpochs = 200,
    optimizer = OPTIM,
    model = model,
    lossFunction = nn.MSELoss(),
    inputTrainCater = tCaterogies,
    inputValidCater = vCaterogies,
    outputTrainPrice = tPrice,
    outputValidPrice = vPrice)

Epoch 200, Training loss 0.0263, Validation loss 0.0199


In [18]:
with torch.cuda.device(0):
  net = model
  macs, paramaters = get_model_complexity_info(net, (1, 5), as_strings=True, print_per_layer_stat=True, verbose=True)
  print('{:<30}  {:<8}'.format('Computational complexity: ', macs))
  print('{:<30}  {:<8}'.format('Number of parameters: ', paramaters))

Sequential(
  0.0 M, 100.000% Params, 0.0 GMac, 100.000% MACs, 
  (0): Linear(0.0 M, 22.967% Params, 0.0 GMac, 22.967% MACs, in_features=5, out_features=8, bias=True)
  (1): Tanh(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, )
  (2): Linear(0.0 M, 68.900% Params, 0.0 GMac, 68.900% MACs, in_features=8, out_features=16, bias=True)
  (3): Tanh(0.0 M, 0.000% Params, 0.0 GMac, 0.000% MACs, )
  (4): Linear(0.0 M, 8.134% Params, 0.0 GMac, 8.134% MACs, in_features=16, out_features=1, bias=True)
)
Computational complexity:       0.0 GMac
Number of parameters:           209     


In [19]:
############ Question 2 #############

In [20]:
############## Part 1 ###############

In [21]:
transforms.Normalize((0.4915, 0.4823, 0.4468),(0.2470, 0.2435, 0.2616))

Normalize(mean=(0.4915, 0.4823, 0.4468), std=(0.247, 0.2435, 0.2616))

In [22]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [23]:
from torchvision import datasets
data_path = 'C:/Users/tycha/OneDrive/Documents/Sarah/Real_Time/FICAR10'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468),(0.2470, 0.2435, 0.2616))]))
cifar10_val = datasets.CIFAR10(data_path, train=False, download=False, transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.4915, 0.4823, 0.4468),(0.2470, 0.2435, 0.2616))]))

Files already downloaded and verified


In [24]:
CIFAR_model = nn.Sequential(
              nn.Linear(3072,512),
              nn.Tanh(),
              nn.Linear(512,10),
              nn.LogSoftmax(dim=1))
#CIFAR_model.to(device)
loss_fn = nn.NLLLoss()

In [25]:
optimizer = optim.SGD(CIFAR_model.parameters(),lr=1e-3)

In [26]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)
n_epochs = 300

In [27]:
for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        #imgs, labels = imgs.to(device), labels.to(device)
        batch_size = imgs.shape[0]
        outputs = CIFAR_model(imgs.view(batch_size, -1))
        loss = loss_fn(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch == 200:
        print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 200, Loss: 0.567221


In [32]:
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        #imgs, labels = imgs.to(device), labels.to(device)
        batch_size = imgs.shape[0]
        outputs = CIFAR_model(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
print("Accuracy: %", (correct/total))

Accuracy: % 0.4968


In [33]:
############## Part 2 ###############

In [34]:
CIFAR_model2 = nn.Sequential(
              nn.Linear(3072,512),
              nn.Tanh(),
              nn.Linear(512,256),
              nn.Tanh(),
              nn.Linear(256,128),
              nn.Tanh(),
              nn.Linear(128,10),
              nn.LogSoftmax(dim=1))
#CIFAR_model2.to(device)
loss_fn = nn.NLLLoss()

In [35]:
optimizer = optim.SGD(CIFAR_model2.parameters(),lr=1e-2)

In [36]:
train_loader = torch.utils.data.DataLoader(cifar10, batch_size=64, shuffle=True)
n_epochs = 300

In [37]:
for epoch in range(n_epochs):
    for imgs, labels in train_loader:
        #imgs, labels = imgs.to(device), labels.to(device)
        batch_size = imgs.shape[0]
        outputs = CIFAR_model2(imgs.view(batch_size, -1))
        loss = loss_fn(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if epoch == 300:
        print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

In [39]:
print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 299, Loss: 0.000346


In [38]:
val_loader = torch.utils.data.DataLoader(cifar10_val, batch_size=64, shuffle=False)

correct = 0
total = 0

with torch.no_grad():
    for imgs, labels in val_loader:
        #imgs, labels = imgs.to(device), labels.to(device)
        batch_size = imgs.shape[0]
        outputs = CIFAR_model2(imgs.view(batch_size, -1))
        _, predicted = torch.max(outputs, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())
        
print("Accuracy: %", (correct/total))

Accuracy: % 0.459
